# Graph Data structures

Topological ordering

In [ ]:
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COUNT(predecessor) AS incomingDependencies
OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)
WITH activity, incomingDependencies, COALESCE(MAX(LENGTH(path)), 0) AS depth
RETURN 
    activity.name AS activityName, 
    activity.description AS description, 
    activity.duration AS duration, 
    depth, 
    incomingDependencies
ORDER BY depth DESC, incomingDependencies ASC


Early start and early finish topological ordering

In [ ]:
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COLLECT(predecessor) AS predecessors
ORDER BY activity.depth ASC
WITH activity,
    REDUCE(maxFinish = 0, pred IN predecessors | 
        CASE 
            WHEN pred.earlyFinish IS NOT NULL AND pred.earlyFinish > maxFinish THEN pred.earlyFinish 
            ELSE maxFinish 
        END) AS earlyStart
SET activity.earlyStart = COALESCE(earlyStart, 0),
    activity.earlyFinish = COALESCE(earlyStart, 0) + activity.duration
RETURN activity.name AS activityName, activity.earlyStart, activity.earlyFinish
ORDER BY activity.depth ASC
